In [ ]:
import torch
import numpy as np

In [ ]:
import cvxpy as cp


x = cp.Variable()
y = cp.Variable()
gamma = cp.Parameter(nonneg=True)
problem = cp.Problem(cp.Minimize(gamma * y), [y == gamma * x])
print("Is DPP? ", problem.is_dcp(dpp=True))
print("Is DCP? ", problem.is_dcp(dpp=False))

In [ ]:
a = torch.tensor([[1],[2]])
b = torch.tensor([[0]])
torch.cat((a,b),0).T
print(b - 1.0)

In [ ]:
class dynamics(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        # x_{t+1} = f(x_t) + g(x_t)u_t
        x = input[0:2,0]
        u = input[2:4,0]
        fx = x
        gx = torch.tensor([[1,0],[0,1]],dtype=torch.float)
        return fx + torch.matmul(gx, u)
    
    @staticmethod
    def backward(ctx, grad_output):
        print("hello")
        input, = ctx.saved_tensors
#         grad_input = grad_output.clone()
        input, = ctx.saved_tensors
        x = input[0:2].detach().numpy()
        u = input[2:4].detach().numpy()
        gx = np.array([[1,0],[0,1]])
        gxu = np.array([[1,0],[0,1]]) @ u
        df_dx =  np.array([[1,0],[0,1]])
        dgxu_dx = np.array([[0,0],[0,0]])
#         print(dg_dx)
        print(u)
        gradient = torch.tensor(np.append(df_dx+dgxu_dx, gx, axis=1),dtype=torch.float)
#         print("gradient",gradient.size())
#         print("grad_output",grad_output.size())
#         print("grad_output",grad_output)
        return torch.reshape(torch.matmul(torch.reshape(grad_output,(1,2)),gradient),(4,1))
        

In [ ]:
dtype = torch.float
device = torch.device("cpu")

dynam = dynamics.apply
x = np.array([1,1]).reshape(-1,1)
u = np.array([5,2]).reshape(-1,1)
input = torch.tensor(np.append(x,u,axis=0),dtype=dtype, requires_grad=True)
y_pred = dynam(input)
print(y_pred)
# loss = y_pred.sum()
# loss.backward()
# print(input.grad)
# print("xx",np.shape(x))

import cvxpy as cp
from cvxpylayers.torch import CvxpyLayer

x = cp.Variable(2)
y = cp.Parameter(2)
objective = cp.Minimize(0.5*cp.norm(x-y/2))
const = [x[0]<=10]
problem = cp.Problem(objective, const)
assert problem.is_dpp()
# print(problem.parameters())

cvxpylayer = CvxpyLayer(problem, parameters=[y], variables=[x])
# this is just y_pred, a_tch = torch.tensor(a.value, requires_grad=True, dtype=torch.float)

# solve the problem
solution, = cvxpylayer(y_pred)

print(solution.detach().numpy())
loss = solution.sum()
print("loss",loss)
loss.backward()
print(input.grad)

In [ ]:
import time
import cvxpy as cp
import numpy as np
from cvxpylayers.torch import CvxpyLayer
import torch
x = cp.Variable((2,1))
a = cp.Parameter((2,1),value=np.array([[.5],[.5]]))
c = cp.Parameter((2,1),value=np.array([[-2],[-2]]))
const = [x<=c]
s = time.time()
objective = cp.Minimize(0.5*cp.norm(x-a))
problem = cp.Problem(objective, const)
assert problem.is_dpp()
print(problem.parameters())

cvxpylayer = CvxpyLayer(problem, parameters=[a, c], variables=[x])
a_tch = torch.tensor(a.value, requires_grad=True, dtype=torch.float)
c_tch = torch.tensor(c.value, requires_grad=True, dtype=torch.float)

# solve the problem
solution, = cvxpylayer(a_tch, c_tch)
print(solution.detach().numpy())
print(solution)
# print(x.value)
e1 = torch.tensor(np.array([1.0,0]), dtype=torch.float)
e2 = torch.tensor(np.array([0.0,1.0]), dtype=torch.float)

# # compute the gradient of the sum of the solution with respect to a, c
# solution.sum().backward()
u1 = torch.matmul(e1,solution)
u2 = torch.matmul(e2,solution)
print("u1",u1)

u1.backward(retain_graph=True)
print("a grad ", a_tch.grad.numpy())
print("c grad ", c_tch.grad.numpy())

c_prev = np.copy(c_tch.grad.numpy())


print("u2",u2)
u2.backward(retain_graph=False)
print(time.time()-s)
print("a grad ", a_tch.grad.numpy().reshape(1,-1))
print("c grad ", c_tch.grad.numpy())
print("c_prev",c_prev)
print("c grad diff", c_tch.grad.numpy()-c_prev)
# problem.solve()
# print(problem.status)
# print(x.value)

In [ ]:
print(solution)

In [ ]:
u = torch.tensor(np.array([[1],[1]]),dtype=dtype, requires_grad=True)
input = torch.cat((solution,u),axis=0)

In [ ]:
y_pred = dynam(input)
print(y_pred)
loss = y_pred.sum()
loss.backward()
print(c_tch.grad)

In [ ]:
a = np.array([0])
b = a

In [ ]:
b[0]=3

In [ ]:
a

In [ ]:
b = np.copy(a)

In [ ]:
b[0]=5

In [ ]:
b

In [ ]:
class robot:
    
    def fx(self,x):
        return x
    
    def gx(self,x):
        return torch.tensor(np.array([[1,0],[0,1]]),dtype=torch.float);
    
    def df_dx(self,x, type = 'tensor'):
        if type == 'numpy':
            return np.array(np.array([[1,0],[0,1]]))
        if type == 'tensor':
            return torch.tensor([[1,0],[0,1]],dtype=torch.float)
        
    def dgxu_dx(self,x, type='tensor'):
        if type == 'numpy':
            return np.array([[0,0],[0,0]])
        if type == 'tensor':
            return torch.tensor([[0,0],[0,0]],dtype=torch.float)
        

In [ ]:
def fx_(x):
    return x

def gx_(x):
    return torch.tensor(np.array([[1,0],[0,1]]),dtype=torch.float);

def df_dx_(x, type = 'tensor'):
    if type == 'numpy':
        return np.array(np.array([[1,0],[0,1]]))
    if type == 'tensor':
        return torch.tensor([[1,0],[0,1]],dtype=torch.float)
    
def dgxu_dx_(x, type='tensor'):
    if type == 'numpy':
        return np.array([[0,0],[0,0]])
    if type == 'tensor':
        return torch.tensor([[0,0],[0,0]],dtype=torch.float)

In [ ]:
class torch_dynamics(torch.autograd.Function):
    
    def __init__(self, robot):
        self.robot = robot

    @staticmethod
    def forward(ctx, input_x, input_u, robot):
#         ctx.save_for_backward(input)
        # x_{t+1} = f(x_t) + g(x_t)u_t
        x = input_x
        u = input_u
        fx = robot.fx(x)  #tensor
        gx = robot.gx(x)  #tensor
        
#         gxu = torch.tensor(robot.gx(x).detach().numpy() @ u.detach().numpy()),dtype=torch.float)
        gxu = torch.matmul(gx, u)
        df_dx =  robot.df_dx(x) # numpy
        dgxu_dx = robot.dgxu_dx(x)  # numpy
        
        print(gx, gxu, df_dx, dgxu_dx)
        
        ctx.save_for_backward(x,u, gx, gxu, df_dx, dgxu_dx)

        return fx + torch.matmul(gx, u)

    @staticmethod
    def backward(ctx, grad_output):

        input_x, input_u, gx, gxu, df_dx, dgxu_dx, = ctx.saved_tensors
        x = input_x.detach().numpy()
        u = input_u.detach().numpy()
        
        print(grad_output)
        
        gradient = torch.tensor(np.append(df_dx+dgxu_dx, gx, axis=1),dtype=torch.float)
        print("gradient",gradient.size())
        print("hello")
        print("grad_output",grad_output.size())
        print("grad_output",grad_output)
        print( torch.reshape(torch.matmul(torch.reshape(grad_output,(1,2)),gradient),(4,1)) )
        return torch.reshape(torch.matmul(torch.reshape(grad_output,(1,2)),gradient),(4,1))

In [ ]:
dtype = torch.float
device = torch.device("cpu")

rob = robot()

dynam = torch_dynamics.apply
x = torch.tensor(np.array([1,1]).reshape(-1,1),dtype=dtype, requires_grad=True)
u = torch.tensor(np.array([3,2]).reshape(-1,1),dtype=dtype, requires_grad=True)
# input = torch.tensor(np.append(x,u,axis=0),dtype=dtype, requires_grad=True)
y_pred = dynam(x,u, rob)
print(y_pred)
loss = y_pred.sum()
loss.backward()
print(input.grad)

xx = x.detach().numpy()
print(np.shape(xx))

In [1]:
import torch
import torch.nn as nn
import numpy as np
from robot_models.Unicycle2D import *
from robot_models.SingleIntegrator import *
from dynamics import *
import time

In [10]:
dynam = torch_dynamics.apply


class closedLoopLayer(nn.Module):

    def __init__(self,agent_class,target_class,alpha1,alpha2,alpha3,k):  # pass in unicycle and target here instead of taking it from globla header declarations
        super().__init__()

        self.agent_nominal_controller = agent_class.nominal_controller_tensor2
        self.target_xdot = target_class.xdotTensor

        self.CLF_loss_tensor = agent_class.CLF_loss_tensor_simple
        self.CBF1_loss_tensor = agent_class.CBF1_loss_tensor_simple
        self.CBF2_loss_tensor = agent_class.CBF2_loss_tensor_simple
        self.CBF3_loss_tensor = agent_class.CBF3_loss_tensor_simple

        self.alpha1_tch = torch.tensor(alpha1, requires_grad=True, dtype=torch.float)
        self.alpha2_tch = torch.tensor(alpha2, requires_grad=True, dtype=torch.float)
        self.alpha3_tch = torch.tensor(alpha3, requires_grad=True, dtype=torch.float)
        self.k_tch = torch.tensor(k, requires_grad=True, dtype=torch.float)

        self.solver_args = {
            'verbose': False,
            'max_iters': 1000000
        }

    def forward(self,x_agent,x_target,u_target):

        U_d_ = self.agent_nominal_controller(x_agent,x_target)
        target_xdot_ = self.target_xdot(x_target,u_target)  ######################### target.U??

        # simplified define tensors for cvx DPP form
        V_,dV_dxA_f_, dV_dxA_g_,dV_dxB_temp = self.CLF_loss_tensor(x_agent,x_target)
        h1_,dh1_dxA_f_, dh1_dxA_g_,dh1_dxB_temp = self.CBF1_loss_tensor(x_agent,x_target)
        h2_,dh2_dxA_f_, dh2_dxA_g_,dh2_dxB_temp = self.CBF2_loss_tensor(x_agent,x_target)
        h3_,dh3_dxA_f_, dh3_dxA_g_,dh3_dxB_temp = self.CBF3_loss_tensor(x_agent,x_target)

        dV_dxB_target_xdot_ = dV_dxB_temp @ target_xdot_
        dh1_dxB_target_xdot_ = dV_dxB_temp @ target_xdot_
        dh2_dxB_target_xdot_ = dV_dxB_temp @ target_xdot_
        dh3_dxB_target_xdot_ = dV_dxB_temp @ target_xdot_

        ah1_ = self.alpha1_tch*h1_  
        ah2_ = self.alpha2_tch*h2_  
        ah3_ = self.alpha3_tch*h3_  
        kV_ = self.k_tch * V_  

        # Find the controller
        solution, = cvxpylayer(U_d_, ah1_,dh1_dxA_f_, dh1_dxA_g_,dh1_dxB_target_xdot_, ah2_,dh2_dxA_f_, dh2_dxA_g_,dh2_dxB_target_xdot_, ah3_,dh3_dxA_f_, dh3_dxA_g_, dh3_dxB_target_xdot_,  kV_,dV_dxA_f_,dV_dxA_g_ ,dV_dxB_target_xdot_, solver_args=self.solver_args)

        # Tensors for constraint derivative
        cbf1 = dh1_dxA_f_ + torch.matmul(dh1_dxA_g_, solution) + dh1_dxB_target_xdot_ + ah1_
        cbf2 = dh2_dxA_f_ + dh2_dxA_g_ @ solution + dh2_dxB_target_xdot_ + ah2_ 
        cbf3 = dh3_dxA_f_ + dh3_dxA_g_ @ solution + dh3_dxB_target_xdot_ + ah3_
        # lyap = -dV_dxA_f_ - dV_dxA_g_ @ solution - dV_dxB_target_xdot_ - kV_ 

        return solution[0], [cbf1, cbf2, cbf3]


class horizonControl(nn.Module):

    def __init__(self,):
        super().__init__()
        self.one_step_forward = closedLoopLayer(Unicycle2D,SingleIntegrator,1.5,1.0,1.0,0.2)  # pass agent and target functions here


    def forward(self,x_agent,x_target):
        '''
            x_agent: initial location of agent
            x_target: array of future target locations
        '''

        # Time 1:
        # s_time = time.time()
        u0, cons0 = self.one_step_forward(x_agent,x_target[0],torch.tensor( np.array([1.0,0.0]).reshape(-1,1),dtype=torch.float ))
        x1 = dynam(x_agent,u0)

        #Time 2: 
        u1, cons1 = self.one_step_forward(x1,x_target[1],torch.tensor( np.array([1.0,0.0]).reshape(-1,1),dtype=torch.float ))
        x2 = dynam(x1,u1)

        #Time 3: 
        u2, cons2 = self.one_step_forward(x2,x_target[2],torch.tensor( np.array([1.0,0.0]).reshape(-1,1),dtype=torch.float ))
        x3 = dynam(x2,u2)

        #Time 4: 
        u3, cons3 = self.one_step_forward(x1,x_target[1],torch.tensor( np.array([1.0,0.0]).reshape(-1,1),dtype=torch.float ))
        x4 = dynam(x1,u3)

        return [x1, x2, x3, x4], \
               [u0, u1, u2, u3], \
               [cons0, cons1, cons2, cons3]

In [11]:
Fx = torch.tensor(np.array([0.0,0.2,0.0]).reshape(-1,1),dtype=torch.float)
Tx = torch.tensor(np.array([1.0,0.0]).reshape(-1,1),dtype=torch.float)

# Test Closed Loop

model = closedLoopLayer(Unicycle2D,SingleIntegrator,1.5,1.0,1.0,0.2)
model(Fx,Tx,torch.tensor( np.array([1.0,0.0]).reshape(-1,1),dtype=torch.float ) )

(tensor([[-10.2958],
         [-43.2691]], grad_fn=<SelectBackward>),
 [tensor([[[10.4182]]], grad_fn=<AddBackward0>),
  tensor([[[0.]]], grad_fn=<AddBackward0>),
  tensor([[[0.]]], grad_fn=<AddBackward0>)])

In [12]:
mpc = horizonControl()
x, u, cons = mpc(Fx,[Tx,Tx,Tx,Tx])

In [16]:
print(x[0])

x[0].sum().backward(retain_graph=True)


tensor([[-0.1030],
        [ 0.2000],
        [-0.4327]], grad_fn=<torch_dynamicsBackward>)


In [22]:
mpc.one_step_forward.alpha3_tch.grad

tensor(2.3364)

In [41]:
class sampleNN(nn.Module):
    
    def __init__(self):
        super().__init__()        
        self.a1 = torch.tensor(2, requires_grad=True, dtype=torch.float)
        
    def forward(self,x):
        return torch.square(self.a1*x)

In [57]:
m = sampleNN()
x = torch.tensor(1.0, requires_grad=True, dtype=torch.float)

In [58]:
y = m(x)
z = m(y)

In [59]:
z

tensor(64., grad_fn=<PowBackward0>)

In [64]:
z.sum().backward(retain_graph=True)

In [65]:
m.a1.grad

tensor(192.)

In [62]:
m.a1.grad = None

In [38]:
2*np.sin( 2*np.sin(np.pi/3) )

1.9740532899807075

In [45]:
2**6

64

In [49]:
6*2**5

192

In [66]:
aa = np.array([1,2,3,4])

In [68]:
np.shape(aa)[0]

4

In [70]:
np.zeros((1,3))

array([[0., 0., 0.]])

In [74]:
a = [2,3,4]
b = [5,6,7]

In [75]:
a = a + b

In [76]:
a

[2, 3, 4, 5, 6, 7]

In [77]:
a.append(8)

In [78]:
a

[2, 3, 4, 5, 6, 7, 8]